In [1]:
"""imports"""
import os
from ansys.edb.database import Database
from ansys.edb.session import launch_session
import ansys.edb as edb
from ansys.edb.definition.djordjecvic_sarkar_model import DjordjecvicSarkarModel
import json
import psutil

In [2]:
"""configs"""
ROOT = os.getcwd()
EXE_ROOT = os.environ['ANSYSEM_ROOT232']
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT,'data','meshed_ground.aedb')
out = os.path.join(ROOT,'data','stackup_export.aedb')
stackup_file = os.path.join(ROOT,'data','stackup_export.json')

In [3]:
"""launch session"""
PROCNAME = "EDB_RPC_Server.exe"
try:
    for proc in psutil.process_iter():
        if proc.name() == PROCNAME:
            proc.kill()
except:
    pass
session = launch_session(EXE_ROOT, 50051)

In [4]:
"""open db"""
db = Database.open(EDB_FILE,False)
cell = db.circuit_cells[0]
layout = cell.layout
layerStats = []
lc = layout.layer_collection
layers = lc.get_layers()
layer_names = [i.name for i in layers]

In [5]:
"""extracting layer information"""
layer_info = []
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material().value
        thickness = i.thickness.value
        lower_elevation = i.lower_elevation.value
        full_material = i.get_fill_material().value
        
        if 'SIGNAL_LAYER' in str(layer_type):
            matmodel = edb.definition.material_def.MaterialDef.find_by_name(db,i.get_material().value)
            Sigma = matmodel.get_property(edb.definition.material_def.MaterialProperty.CONDUCTIVITY).value
            x = (name,
            layer_type,
            material,
            Sigma,
            thickness,
            lower_elevation,
            full_material)                
        elif 'DIELECTRIC_LAYER' in str(layer_type):
            matmodel = edb.definition.material_def.MaterialDef.find_by_name(db,i.get_material().value)
            Td = matmodel.get_property(edb.definition.material_def.MaterialProperty.DIELECTRIC_LOSS_TANGENT).value
            Er = matmodel.get_property(edb.definition.material_def.MaterialProperty.PERMITTIVITY).value
            x = (name,
            layer_type,
            material,
            Er,
            Td,
            thickness,
            lower_elevation,
            full_material)
        layer_info.append(x)
db.save_as(out)

In [6]:
"""Write layer information to json file"""
with open(stackup_file, 'w') as fp:
    json.dump(layer_info, fp,indent=' ')

In [7]:
"""End the session"""
session.disconnect()